In [ ]:
# !pip install -r requirements.txt

In [ ]:

from EMG_TFC_LTCR import COMET
from models.encoder import FTClassifier
# import datautils


from tasks.fine_tuning_UCR_not_balance import finetune_fit
from tasks.fine_tuning_UCR_not_balance import finetune_predict
from tasks.fine_tuning_UCR_not_balance import finetune_predict
from tasks.linear_evaluation_UCR_not_balance import eval_classification
# from dataloading.ad_preprocessing import load_ad
from config_files.EMG_Configs_my import Config as Configs
from dct_func import FFT_for_Period
import os
import torch
import numpy as np
import pandas as pd
import torch.nn as nn
import matplotlib.pyplot as plt
import random
import copy
import sklearn
from utils_cpc import plot_channels #
from utils_cpc import process_batch_ts
from utils_cpc import split_data_label
from utils_cpc import start_logging
from utils_cpc import stop_logging
from utils_cpc import seed_everything

from datetime import datetime

In [ ]:
configs = Configs()
RANDOM_SEED = configs.RANDOM_SEED

# Autoload the modified python file
%reload_ext autoreload
%autoreload 2

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

#### Specify saving and logging directory

In [ ]:
working_directory = configs.working_directory
dataset_save_path = working_directory
if not os.path.exists(working_directory):
    os.makedirs(working_directory)

logging_directory = configs.logging_directory
if not os.path.exists(logging_directory):
    os.makedirs(logging_directory)

#### Load and preprocessing data

In [ ]:
# from TSC_dataloader import TSC_multivariate_data_loader, TSC_data_loader_ts
from OS_CNN_utils.dataloader.TSC_data_loader import TSC_multivariate_data_loader,TSC_data_loader

In [ ]:

datasetPath = "datasets/"

In [ ]:

dataset = "NerveDamage"

In [ ]:

# X_train, y_train, X_test, y_test = TSC_data_loader(datasetPath, dataset)
# X_train = X_train.reshape(X_train.shape[0],1,-1)
# X_test = X_test.reshape(X_test.shape[0],1,-1)

In [ ]:
X_train, y_train, X_test, y_test = TSC_multivariate_data_loader(datasetPath, dataset)

In [ ]:
X_train.shape

In [ ]:
y_train.shape

In [ ]:
X_test.shape


In [ ]:
from sklearn.utils import shuffle
import numpy as np
import os


def split_ucr(X_train, X_test, y_train, y_test,ratio = 0.2):


    # 存储文件名
    filenames = []
    train_len = int(X_train.shape[0] * (1-ratio))
    val_len = X_train.shape[0] - train_len
    # reshape and shuffle

    X_train, y_train = shuffle(X_train, y_train, random_state=42)

    X_test, y_test = shuffle(X_test, y_test, random_state=42)
    X_val = X_train[train_len:]
    y_val = y_train[train_len:]
    X_train = X_train[:train_len]
    y_train = y_train[:train_len]
    # print(y_val.shape)

    user_train = np.ones((y_train.shape[0], 1))
    user_val = np.ones((y_val.shape[0], 1))
    user_test = np.ones((y_test.shape[0], 1))
    # print(y_val.shape)
    # print(user_val.shape,y_val.reshape(y_val.shape[0],1).shape)
    y_train = np.hstack((y_train.reshape(y_train.shape[0],1), user_train))
    y_val = np.hstack((y_val.reshape(y_val.shape[0],1), user_val))
    y_test = np.hstack((y_test.reshape(y_test.shape[0],1), user_test))

    return X_train, X_val, X_test, y_train, y_val, y_test



In [ ]:
X_train = X_train.reshape(X_train.shape[0], X_train.shape[2], X_train.shape[1])
# X_test = X_test.reshape(205, 18530, 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[2], X_test.shape[1])

In [ ]:
X_train.shape

In [ ]:
periodicity, freq_list = FFT_for_Period(torch.tensor(X_train), 500) # 
periodicity = torch.from_numpy(np.array([periodicity]))
# periodicity = periodicity.item()
periodicity
# tensor([[1280,  640,  426,  320,  256,  213,  182,  160]])

In [ ]:
X_train.shape

In [ ]:
plot_channels(1, X_train[1][:,:])

In [ ]:
plot_channels(1, X_train[2][:,:])

In [ ]:
plot_channels(1, X_train[3][:,:])

In [ ]:
X_train, X_val, X_test, y_train, y_val, y_test = split_ucr(X_train, X_test, y_train, y_test,ratio = 0.2)

In [ ]:
print(X_train.shape)
print(X_val.shape)
print(X_test.shape)
print(y_train.shape)
print(y_val.shape)
print(y_test.shape)

In [ ]:
# data_path = "datasets/AbnormalHeartbeat/Feature/"
# label_path = "datasets/AbnormalHeartbeat/label.npy"
# val_ids = [17,18]  # specify patient ID for validation and test set
# test_ids = [19,20]
# X_trial_train, X_trial_val, X_trial_test, y_trial_train, y_trial_val, y_trial_test = load_ad(val_ids, test_ids, data_path, label_path)
# print(X_trial_train.shape)
# print(y_trial_train.shape)
# print(X_trial_val.shape)
# print(y_trial_val.shape)
# print(X_trial_test.shape)
# print(y_trial_test.shape)

In [ ]:
# normalize data
X_trial_train = process_batch_ts(X_train, normalized=True, bandpass_filter=False)
X_trial_val = process_batch_ts(X_val, normalized=True, bandpass_filter=False)
X_trial_test = process_batch_ts(X_test, normalized=True, bandpass_filter=False)
print(X_trial_train.shape)
print(X_trial_val.shape)
print(X_trial_test.shape)
y_trial_train = y_train
y_trial_val = y_val
y_trial_test = y_test
print(y_trial_train.shape)
print(y_trial_val.shape)
print(y_trial_test.shape)


In [ ]:
# Split trail-level data into sample-level data
X_train, y_train = split_data_label(X_trial_train,y_trial_train, sample_timestamps=configs.S_TIMESTAMPS, overlapping=configs.S_OVERLAPPING)
X_val, y_val = split_data_label(X_trial_val,y_trial_val, sample_timestamps=configs.S_TIMESTAMPS, overlapping=configs.S_OVERLAPPING)
X_test, y_test = split_data_label(X_trial_test,y_trial_test, sample_timestamps=configs.S_TIMESTAMPS, overlapping=configs.S_OVERLAPPING)
print(X_train.shape)
print(X_val.shape)
print(X_test.shape)
print(y_train.shape)
print(y_val.shape)
print(y_test.shape)

In [ ]:
del X_trial_train
del X_trial_val
del X_trial_test

In [ ]:
# # plot first sample, first four channels
# plot_channels(1, X_trial_train[0][:,:])

In [ ]:
# torch.__version__


In [ ]:
import torch

print("PyTorch version:", torch.__version__)
print("GPU available:", torch.cuda.is_available())

print("cuDNN available:", torch.backends.cudnn.enabled)


## Train model

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"The program will run on {device}!")
# device = "cuda"

### Callback functions

In [ ]:
def pretrain_callback(model, loss):
    n = model.n_epochs
    metrics_dict = {}
    if n % 1 == 0:
        metrics_dict = eval_classification(model, X_train, y_train[:, 0], X_val, y_val[:, 0], fraction=1)
        print(metrics_dict)
        model.save(f"{working_directory}seed{RANDOM_SEED}_pretrain_model.pt")
    return metrics_dict['F1']

In [ ]:
def finetune_callback(model, f1, fraction=1.0):
    n = model.n_epochs
    if model.n_epochs == 1:
        model.finetune_f1 = f1
        torch.save(model.state_dict(), f"{working_directory}seed{RANDOM_SEED}_max_f1_{fraction}_finetune_model.pt")
    # control the saving frequency
    if n % 1 == 0:
        if f1 > model.finetune_f1:
            model.finetune_f1 = f1
            torch.save(model.state_dict(), f"{working_directory}seed{RANDOM_SEED}_max_f1_{fraction}_finetune_model.pt")
    return finetune_callback

## Self-Supervised

In [ ]:
start_time = datetime.now()
# Train a COMET model
seed_everything(RANDOM_SEED)
model = COMET(
    input_dims=configs.input_dims,
    device=device,
    lr=configs.pretrain_lr,
    depth=configs.depth,
    batch_size=configs.pretrain_batch_size,
    # batch_size=512, #simclr 
    output_dims=configs.output_dims,
    flag_use_multi_gpu=configs.flag_use_multi_gpu,
    after_epoch_callback=pretrain_callback,
)

epoch_loss_list, epoch_f1_list = model.fit(
    X_train,
    y_train,
    shuffle_function = configs.shuffle_function,
    verbose=configs.verbose,
    n_epochs=configs.n_epochs,
    masks = configs.masks,
    factors = configs.factors
)

end_time = datetime.now()
print(f'Duration: {end_time - start_time}')

                                test

## Linear evaluation

In [ ]:
start_time = datetime.now()
seed_everything(RANDOM_SEED)
pretrain_model = COMET(
    input_dims=configs.input_dims,
    device=device,
    lr=configs.pretrain_lr,
    depth=configs.depth,
    batch_size=configs.pretrain_batch_size,
    output_dims=configs.output_dims,
    flag_use_multi_gpu=configs.flag_use_multi_gpu,
    after_epoch_callback=pretrain_callback,
)

pretrain_model.load(f"{working_directory}seed{RANDOM_SEED}_pretrain_model.pt")

start_logging(RANDOM_SEED, logging_directory)
val_metrics_dict = eval_classification(pretrain_model, X_train, y_train[:, 0], X_val, y_val[:, 0])
print("Linear evaluation for validation set\n",val_metrics_dict)
test_metrics_dict = eval_classification(pretrain_model, X_train, y_train[:, 0], X_test, y_test[:, 0])
print("Linear evaluation for test set\n",test_metrics_dict)
print()
stop_logging()

## Full fine-tuning

In [ ]:
seed_everything(RANDOM_SEED)
finetune_model = FTClassifier(input_dims=configs.input_dims, output_dims=configs.output_dims, depth=configs.depth, p_output_dims=configs.num_classes, device=device, flag_use_multi_gpu=configs.flag_use_multi_gpu)
finetune_model.net.load_state_dict(torch.load(f"{working_directory}seed{RANDOM_SEED}_pretrain_model.pt"))

In [ ]:
seed_everything(RANDOM_SEED)
epoch_loss_list, epoch_f1_list = finetune_fit(finetune_model, X_train, y_train[:, 0], X_val, y_val[:, 0], batch_size=configs.finetune_batch_size_100, finetune_epochs=configs.finetune_epochs_100, num_classes=configs.num_classes, finetune_lr=configs.finetune_lr_100, fraction=configs.fraction_100, device=device, callback=finetune_callback)

In [ ]:
finetune_model.load_state_dict(torch.load(f"{working_directory}seed{RANDOM_SEED}_max_f1_{configs.fraction_100}_finetune_model.pt"))
start_logging(RANDOM_SEED, logging_directory)
print(f"Finetune for {configs.fraction_100} fraction of validation set")
finetune_predict(finetune_model, X_val, y_val[:,0],num_classes = configs.num_classes)
print(f"Finetune for {configs.fraction_100} fraction of test set")
finetune_predict(finetune_model, X_test, y_test[:,0],num_classes = configs.num_classes)
print()
stop_logging()

In [ ]:
plt.figure(1, figsize=(8, 8))
plt.subplot(121)
plt.plot(epoch_loss_list)
plt.title('Loss')
plt.subplot(122)
plt.plot(epoch_f1_list)
plt.title('Accuracy')
plt.show()

## Semi-supervised

### 10% label

In [ ]:
seed_everything(RANDOM_SEED)
finetune_model = FTClassifier(input_dims=configs.input_dims, output_dims=configs.output_dims, depth=configs.depth, p_output_dims=configs.num_classes, device=device, flag_use_multi_gpu=configs.flag_use_multi_gpu)
finetune_model.net.load_state_dict(torch.load(f"{working_directory}seed{RANDOM_SEED}_pretrain_model.pt"))

In [ ]:
seed_everything(RANDOM_SEED)
epoch_loss_list, epoch_f1_list = finetune_fit(finetune_model, X_train, y_train[:, 0], X_val, y_val[:, 0], batch_size=configs.finetune_batch_size_10,finetune_epochs=configs.finetune_epochs_10, num_classes=configs.num_classes, finetune_lr=configs.finetune_lr_10, fraction=configs.fraction_10, device=device, callback=finetune_callback)

In [ ]:
finetune_model.load_state_dict(torch.load(f"{working_directory}seed{RANDOM_SEED}_max_f1_{configs.fraction_10}_finetune_model.pt"))
start_logging(RANDOM_SEED, logging_directory)
print(f"Finetune for {configs.fraction_10} fraction of validation set")
finetune_predict(finetune_model, X_val, y_val[:,0],num_classes = configs.num_classes)
print(f"Finetune for {configs.fraction_10} fraction of test set")
finetune_predict(finetune_model, X_test, y_test[:,0],num_classes = configs.num_classes)
print()
stop_logging()

In [ ]:
plt.figure(1, figsize=(8, 8))
plt.subplot(121)
plt.plot(epoch_loss_list)
plt.title('Loss')
plt.subplot(122)
plt.plot(epoch_f1_list)
plt.title('Accuracy')
plt.show()

### 1% label

In [ ]:
seed_everything(RANDOM_SEED)
finetune_model = FTClassifier(input_dims=configs.input_dims, output_dims=configs.output_dims, depth=configs.depth, p_output_dims=configs.num_classes, device=device, flag_use_multi_gpu=configs.flag_use_multi_gpu)
finetune_model.net.load_state_dict(torch.load(f"{working_directory}seed{RANDOM_SEED}_pretrain_model.pt"))

In [ ]:
seed_everything(RANDOM_SEED)
epoch_loss_list, epoch_f1_list = finetune_fit(finetune_model, X_train, y_train[:, 0], X_val, y_val[:, 0], batch_size=configs.finetune_batch_size_1,finetune_epochs=configs.finetune_epochs_1, num_classes=configs.num_classes, finetune_lr=configs.finetune_lr_1, fraction=configs.fraction_1, device=device, callback=finetune_callback)

In [ ]:
finetune_model.load_state_dict(torch.load(f"{working_directory}seed{RANDOM_SEED}_max_f1_{configs.fraction_1}_finetune_model.pt"))
start_logging(RANDOM_SEED, logging_directory)
print(f"Finetune for {configs.fraction_1} fraction of validation set")
finetune_predict(finetune_model, X_val, y_val[:,0],num_classes = configs.num_classes)
print(f"Finetune for {configs.fraction_1} fraction of test set")
finetune_predict(finetune_model, X_test, y_test[:,0],num_classes = configs.num_classes)
print()
stop_logging()

In [ ]:
plt.figure(1, figsize=(8, 8))
plt.subplot(121)
plt.plot(epoch_loss_list)
plt.title('Loss')
plt.subplot(122)
plt.plot(epoch_f1_list)
plt.title('Accuracy')
plt.show()

In [ ]:
end_time = datetime.now()
print(f'Duration: {end_time - start_time}')